# Algorithme de Pathfinding A*
A* (A-Star) est un algorithme heuristique de recherche de chemin qui trouve le chemin le plus court entre deux points tout en évitant les obstacles.

### Comment fonctionne A* ?
- Le coût réel (g) : la distance déjà parcourue depuis le point de départ.
- Le coût estimé (h) : une estimation de la distance restante jusqu'à l’objectif.
- Le coût total (f) : la somme des deux précédents (f = g + h)



### L’algorithme suit les étapes suivantes :
- Initialisation : On place le point de départ dans une liste ouverte (open list).
- Exploration :
    On choisit la case avec le coût f le plus bas.
    On cherche ses voisins (cases adjacentes).
    On met à jour leurs coûts g, h et f.
    Si un voisin est déjà visité avec un meilleur coût, on l’ignore.
- Résultat :
    Objectif atteint : Si l’objectif est trouvé, on reconstruit le chemin en remontant les parents des cases.
    Pas de solution : Si la liste ouverte est vide avant d'atteindre l'objectif, il n’existe pas de chemin.
    


### Pourquoi A* est efficace ?
Il évite d’explorer inutilement des zones non pertinentes rapidement.

### Utilisation dans le projet
- Dans les jeux de stratégie ou de type Tower Defense, A* est utilisé pour :
    Faire déplacer les ennemis vers la base du joueur en évitant les obstacles.
    Optimiser l’itinéraire des soldats pour minimiser le temps de parcours.
    

## Visuel de la recherche du meilleur chemin
- Dans une grille de 5 x 5
- Couleurs :
    - Cases noires sont des murs
    - Cases bleues sont en attente d'exploration
    - Cases grises sont déjà explorées
    - Case verte est le point courant
    - Case rouge est le but
    - Chemin jaune est le plus court qui ai été trouvé



### Exploration de la grille
- À chaque fois :
    - L'algorithme sélectionne la case avec le coût total (f = g + h) le plus bas.
    - Il marque la case explorée en gris.
    - Il ajoute les cases voisines valides dans la liste des cases à explorer (bleues).

### Trouver le chemin optimal
- Lorsqu'il atteint le but (G dans la grille, case rouge), l'algorithme remonte à travers les cases explorées pour tracer le chemin final.
- Le chemin est dessiné en jaune, la route la plus rapide.

### Affichage dynamique
Chaque étape est affichée progressivement à l'aide de la barre espace, pour que l'on puisse suivre le progrès.

In [ ]:
import pygame
import heapq


# Couleurs
BLANC = (255, 255, 255)
NOIR = (0, 0, 0)
VERT = (0, 255, 0)
ROUGE = (255, 0, 0)
BLEU = (0, 0, 255)
GRIS = (200, 200, 200)

# Définir la grille
grille = [
    ['S', '.', '#', '.', '.'],
    ['.', '.', '.', '.', '.'],
    ['.', '.', '#', '#', '.'],
    ['.', '#', '.', '.', '.'],
    ['.', '.', '.', '.', 'G'],
]

# Paramètres de la grille
TAILLE_GRILLE = 5  
TAILLE_CELLULE = 100 
TAILLE_FENETRE = TAILLE_GRILLE * TAILLE_CELLULE
ecran = pygame.display.set_mode((TAILLE_FENETRE, TAILLE_FENETRE))

# Initialisation de Pygame
pygame.init()
ecran = pygame.display.set_mode((TAILLE_FENETRE, TAILLE_FENETRE))
pygame.display.set_caption("Visualisation du Pathfinding A*")

# Classe du nœud A*
class Case:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.g = float('inf')  # Coût depuis le début
        self.h = 0  # Estimation du coût vers la fin
        self.f = float('inf')  # g + h
        self.parent = None  # Pour reconstruire le chemin

    def __lt__(self, autre):
        return self.f < autre.f  # Tri par priorité

def heuristique(a, b):
    """Fonction de distance heuristique"""
    return abs(a.x - b.x) + abs(a.y - b.y)

def obtenir_voisins(case):
    """Retourne les voisins valides"""
    directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Directions possibles
    # ajouter (1, 1), (-1, 1), (1, -1), (-1, -1) si aller en diagonales
    voisins = []
    
    for dx, dy in directions:
        x, y = case.x + dx, case.y + dy
        if 0 <= x < TAILLE_GRILLE and 0 <= y < TAILLE_GRILLE and grille[x][y] != "#":
            voisins.append(Case(x, y))
    
    return voisins

def a_star(depart, but):
    """Affichage du pathfinding avec un pas manuel"""
    liste_ouverte = []
    heapq.heappush(liste_ouverte, depart)
    depart.g = 0
    depart.h = heuristique(depart, but)  # Calcul de l'heuristique pour la case de départ
    depart.f = depart.g + depart.h  # Mise à jour du coût total f

    liste_fermee = set()
    
    while liste_ouverte:
        ecran.fill(BLANC)  # Effacer l'écran
        dessiner_grille()

        courant = heapq.heappop(liste_ouverte)

        if (courant.x, courant.y) == (but.x, but.y):
            return reconstruire_chemin(courant)

        liste_fermee.add((courant.x, courant.y))

        for voisin in obtenir_voisins(courant):
            if (voisin.x, voisin.y) in liste_fermee:
                continue

            temp_g = courant.g + 1  # coût entre les cases = 1

            # Vérifier si le chemin vers le voisin est meilleur
            if temp_g < voisin.g:
                voisin.parent = courant
                voisin.g = temp_g
                voisin.h = heuristique(voisin, but)  # Recalculer h pour chaque voisin
                voisin.f = voisin.g + voisin.h  # Mettre à jour f (g + h)

                # Mettre à jour la liste
                if voisin not in liste_ouverte:
                    heapq.heappush(liste_ouverte, voisin)

        # Visualisation des couleurs
        dessiner_cases(liste_ouverte, liste_fermee, courant)
        pygame.display.update()

        # Attendre que l'utilisateur appuie sur ESPACE pour continuer
        attendre_barre_espace()

    return None  # Si aucun chemin trouvé

def attendre_barre_espace():
    """Pause l'exécution jusqu'à ce que l'utilisateur appuie sur la touche ESPACE"""
    attente = True
    while attente:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                attente = False


def reconstruire_chemin(case):
    """Reconstruire le chemin à partir des parents des nœuds"""
    chemin = []
    while case:
        chemin.append((case.x, case.y))
        case = case.parent
    return chemin[::-1]  # Chemin inversé

def dessiner_grille():
    """Dessiner la grille à l'écran"""
    for x in range(TAILLE_GRILLE):
        for y in range(TAILLE_GRILLE):
            rect = pygame.Rect(x * TAILLE_CELLULE, y * TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE)
            pygame.draw.rect(ecran, NOIR, rect, 1)

            if grille[x][y] == "#":
                pygame.draw.rect(ecran, NOIR, rect)  # Obstacles
            elif grille[x][y] == "S":
                pygame.draw.rect(ecran, VERT, rect)  # Départ
            elif grille[x][y] == "G":
                pygame.draw.rect(ecran, ROUGE, rect)  # But

def dessiner_cases(liste_ouverte, liste_fermee, courant):
    """Colorier et afficher les coûts des nœuds en cours d'exploration"""
    font = pygame.font.Font(None, 20) 

    for case in liste_fermee:
        rect = pygame.Rect(case[0] * TAILLE_CELLULE, case[1] * TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE)
        pygame.draw.rect(ecran, GRIS, rect)  # Cases explorés

        # Afficher le coût g
        texte = font.render(f"", True, NOIR)
        ecran.blit(texte, (case[0] * TAILLE_CELLULE + 5, case[1] * TAILLE_CELLULE + 5))

    for case in liste_ouverte:
        rect = pygame.Rect(case.x * TAILLE_CELLULE, case.y * TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE)
        pygame.draw.rect(ecran, BLEU, rect)  # Cases à explorer

        # Afficher les coûts g, h, f
        texte_g = font.render(f"g:{case.g}", True, NOIR)
        texte_h = font.render(f"h:{case.h}", True, NOIR)
        texte_f = font.render(f"f:{case.f}", True, NOIR)

        ecran.blit(texte_g, (case.x * TAILLE_CELLULE + 5, case.y * TAILLE_CELLULE + 5))
        ecran.blit(texte_h, (case.x * TAILLE_CELLULE + 5, case.y * TAILLE_CELLULE + 20))
        ecran.blit(texte_f, (case.x * TAILLE_CELLULE + 5, case.y * TAILLE_CELLULE + 35))

    # Case actuel
    rect = pygame.Rect(courant.x * TAILLE_CELLULE, courant.y * TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE)
    pygame.draw.rect(ecran, VERT, rect)

    # Afficher les coûts
    texte_g = font.render(f"g:{courant.g}", True, NOIR)
    texte_h = font.render(f"h:{courant.h}", True, NOIR)
    texte_f = font.render(f"f:{courant.f}", True, NOIR)

    ecran.blit(texte_g, (courant.x * TAILLE_CELLULE + 5, courant.y * TAILLE_CELLULE + 5))
    ecran.blit(texte_h, (courant.x * TAILLE_CELLULE + 5, courant.y * TAILLE_CELLULE + 20))
    ecran.blit(texte_f, (courant.x * TAILLE_CELLULE + 5, courant.y * TAILLE_CELLULE + 35))

def dessiner_chemin(chemin):
    """Dessiner le chemin final."""
    for x, y in chemin:
        rect = pygame.Rect(x * TAILLE_CELLULE, y * TAILLE_CELLULE, TAILLE_CELLULE, TAILLE_CELLULE)
        pygame.draw.rect(ecran, (255, 255, 0), rect)  # Chemin (jaune)

# Trouver les positions de départ et d'arrivée
debut_pos, but_pos = None, None
for x in range(TAILLE_GRILLE):
    for y in range(TAILLE_GRILLE):
        if grille[x][y] == "S":
            debut_pos = (x, y)
        elif grille[x][y] == "G":
            but_pos = (x, y)

debut = Case(*debut_pos)
but = Case(*but_pos)

# Exécution
chemin = a_star(debut, but)

# Afficher le chemin final
if chemin:
    dessiner_chemin(chemin)
    pygame.display.update()

# Boucle de jeu
en_cours = True
while en_cours:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            en_cours = False

pygame.quit()

# Conclusion #
- A* trouve le chemin le plus court de manière efficace.
- L'algorithme explore les chemins et trouve la route la plus rapide.
- Contrairement à d'autres algorithmes comme Dijkstra, A* évite d'examiner inutilement des chemins peu pertinents.